Constrained matrix factorization for ROI extraction deconvolution
-----------------------------------------------------------------
-----------------------------------------------------------------

Import packages
---------------
---------------

Import required packages for analysis
-------------------------------------

In [ ]:
# import required packages for analysis
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')    
except:
    print 'NOT IPYTHON'

import sys
import numpy as np
import ca_source_extraction as cse
from matplotlib import pyplot as plt
from time import time
from scipy.sparse import coo_matrix
import tifffile
import subprocess
import time as tm
import pylab as pl
from time import time
import psutil
import calblitz as cb

Import required packages for visualization
------------------------------------------

In [ ]:
import bokeh.plotting as bpl
from bokeh.io import vform,hplot,vplot,gridplot
from bokeh.models import CustomJS, ColumnDataSource, Slider
from IPython.display import display, clear_output
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np

bpl.output_notebook()

Importing and visualizing movie
-------------------------------
-------------------------------

Load and motion correct
-----------------------
Parameters:

- movie name
- motion corrected movie name
- frame rate
- maximum allowed frame shift
- whether or not to perform again pre processing

In [ ]:
movie_name='movies/demo_mc.tif'
movie_mc_name='movies/demo_mc.hdf5'
frate=30
max_shift_w,max_shift_h=10,10
preprocess=1

if preprocess:
    t1 = time()
    Yr=cb.load(movie_name,fr=frate)
    Yr=Yr-np.min(Yr)                          # make movie positive    
    Yr,shifts,xcorrs,template=Yr.motion_correct(max_shift_w=max_shift_w, max_shift_h=max_shift_h,  method='opencv') 
    max_h,max_w= np.max(shifts,axis=0)
    min_h,min_w= np.min(shifts,axis=0)
    Yr=Yr.crop(crop_top=max_h,crop_bottom=-min_h+1,crop_left=max_w,crop_right=-min_w,crop_begin=0,crop_end=0)
    Yr.save(movie_mc_name)        
    Yr = np.transpose(Yr,(1,2,0)) 
    d1,d2,T=Yr.shape
    Yr=np.reshape(Yr,(d1*d2,T),order='F')
    np.save('Yr',np.asarray(Yr))
    print time() - t1
    clear_output(wait=True)
    print('DONE!')

Memory mapping data
-------------------

In order to reduce the memory usage files are **memory mapped**. They are not loaded into memory unless it is strictly necesary to do so. 

In [ ]:
_,d1,d2=np.shape(cb.load(movie_mc_name,subindices=range(3)))
Yr=np.load('Yr.npy',mmap_mode='r')  
d,T=Yr.shape      
Y=np.reshape(Yr,(d1,d2,T),order='F')
clear_output(wait=True)
print('DONE!')

Play movie
-----------------------

One can play the movie (for the moment at low frame rate)

In [ ]:
m=cb.load(movie_mc_name)
m=m-np.percentile(m,1)
m.play(fr=50,gain=5)

Visualize average movie
-----------------------

One can *visualize and interact* with heatplots

In [ ]:
mean_movie=np.mean(Y,axis=-1)
bpl.show(cse.nb_imshow(mean_movie))

Visualize neurons via neighbouring correlation analysis
-------------------------------------------------------

In [ ]:
Cn = cse.utilities.local_correlations(Y)
bpl.show(cse.nb_imshow(Cn))

Setting up parameters for the algorithm 
---------------------------------------------------------------
---------------------------------------------------------------

Setting parameters for the server and the order of the AR mode
--------------------------------------------------------------
*Main* parameters required for running the algorithm: 
- **p** represents the order of the autoregressive model (p=1 single exponential)
- **gSig** should be half the extension of the average neuron along x and y directions (gSig=[7,7] means the neurons normally fit withon a 15x15 pixels square)
- **K** expected number of neurons. Try to give an estimate per excess
- **ssub** downsampling factor along x and y when datasets are very large, only used in the initialization!
- **n_processes** is the number of parallel processes used to perform analysis
- **deconvolution_strictness** parameters controlling the estimation process of the time constants (higher, up to 1, values in general for sparser activity)

In [ ]:
n_processes = np.maximum(psutil.cpu_count() - 2,1) # roughly number of cores on your machine minus 1
p=2 # order of the AR model (in general 1 or 2)
gSig=[7,7]
K=10
ssub=1
deconvolution_strictness=0.96
options = cse.utilities.CNMFSetParms(Y,p=p,gSig=gSig,K=K,ssub=ssub)
options['temporal_params']['fudge_factor'] = deconvolution_strictness

Firing up the cluster
---------------------

The suite is specially optimized to exploit massive parallelization. We start a local cluster on the machine. 
(Check the activity monitor)

In [ ]:
sys.stdout.flush()  
cse.utilities.stop_server() # trying to stop in case it was already runnning
cse.utilities.start_server(options['spatial_params']['n_processes'])

Initializing the solution
--------------------------
--------------------------

preprocess_data
---------------
- Remove invalid values **Yr** (experimental)
- Estimate noise level per each pixel **sn**
- Estimate parameters of th autoregressive model (related to raise and decay time) **g**

initialize_components
---------------------
- Estimate spatial filters **Atmp**
- Estimate calcium traces **Ctmp**
- Estimate background component **b_in**
- Estimate background time course **f_in** 
- **center** are simply the center of the spatial filters 


In [ ]:
t1 = time()
Yr,sn,g=cse.pre_processing.preprocess_data(Yr,**options['preprocess_params'])
Atmp, Ctmp, b_in, f_in, center=cse.initialization.initialize_components(Y, **options['init_params'])                                                    
print time() - t1 
clear_output(wait=True)
print('DONE!')

Manually refine the solution
----------------------------

Using neighboring correlation as a reference, add components the algorithm might have missed in the initialization.
Click on the center of the identified neurons, the algorithm will automatically estimate the neuron spatial extension. 

Hit enter when you are happy with the selected neuron, then close the window 

In [ ]:
refine_components=True
if refine_components:
    Ain,Cin = cse.utilities.manually_refine_components(Y,options['init_params']['gSig'],coo_matrix(Atmp),Ctmp,Cn,thr=0.9)
else:
    Ain,Cin = Atmp, Ctmp

Visualize and inspect the refined components
--------------------------------------------

If you are not happy, repeat previous step

In [ ]:
p=cse.nb_plot_contour(Cn,Ain,d1,d2,thr=0.9,face_color=None, line_color='black',alpha=0.4,line_width=2)
bpl.show(p)

Refinement of the components based on nonnegative matrix factorization
----------------------------------------------------------------------
----------------------------------------------------------------------



Refinement of spatial components
---------------------------------
At this step, the algorithm tries to estimate the minimal number of pixels required to explain the observed fluorescence signal at each pixel 

In [ ]:
#%%
t1 = time()
A,b,Cin,f_in = cse.spatial.update_spatial_components(Yr, Cin, f_in, Ain, sn=sn, **options['spatial_params'])
t_elSPATIAL = time() - t1
print t_elSPATIAL 
#clear_output(wait=True)
print('DONE!')

Visualize refined components
----------------------------

In [ ]:
p=cse.nb_plot_contour(Cn,A.todense(),d1,d2,thr=0.9,face_color=None, line_color='black',alpha=0.4,line_width=2)
bpl.show(p)

Refinement of temporal component 
---------------------------------

At this step, the algorithm tries to estimate the minimal number of spikes required to explain the observed fluorescence signal at each pixel. The first time in order to obtain a coarse solution the time component are estimated with p=0, equivalent to unconstrained nonnnegative matrix factorization

In [ ]:
t1 = time()
options['temporal_params']['p'] = 0 # set this to zero for fast updating without deconvolution
C,f,S,bl,c1,neurons_sn,g,YrA = cse.temporal.update_temporal_components(Yr,A,b,Cin,f_in,bl=None,c1=None,sn=None,g=None,**options['temporal_params'])
t_elTEMPORAL2 = time() - t1
clear_output(wait=True)
print('DONE!')
print t_elTEMPORAL2

Merging components
------------------

The algorithm might have splitted single neurons in multiple components. The algorithm tries to merge components that are highly correlated and spatially contiguous

In [ ]:
#%% merge components corresponding to the same neuron
t1 = time()
A_m,C_m,nr_m,merged_ROIs,S_m,bl_m,c1_m,sn_m,g_m=cse.merging.merge_components(Yr,A,b,C,f,S,sn,options['temporal_params'], options['spatial_params'], bl=bl, c1=c1, sn=neurons_sn, g=g, thr=0.8, mx=50, fast_merge = True)
t_elMERGE = time() - t1
clear_output(wait=True)
print('DONE!')
print t_elMERGE 
print 'Merged Components'+str(merged_ROIs)

Second iteration refining both spatial and temporal components 
------------------------------------------------------------------

In [ ]:
#refine spatial and temporal components
t1 = time()
A2,b2,C2,f = cse.spatial.update_spatial_components(Yr, C_m, f, A_m, sn=sn, **options['spatial_params'])
C2,f2,S2,bl2,c12,neurons_sn2,g21,YrA = cse.temporal.update_temporal_components(Yr,A2,b2,C2,f,bl=None,c1=None,sn=None,g=None,**options['temporal_params'])
clear_output(wait=True)
print('DONE!')
print time() - t1

Visualize the result
--------------------
--------------------

Components and neighboring correlation image
-----------------------------------

In [ ]:
A_or, C_or, srt = cse.utilities.order_components(A2,C2)
p=cse.utilities.nb_plot_contour(Cn,A_or,d1,d2,thr=0.9,face_color='purple', line_color='black',alpha=0.3,line_width=2)
bpl.show(p)

Components and traces
---------------------

In [ ]:
traces_fluo=cse.utilities.nb_view_patches(Yr,A_or,C_or,b2,f2,d1,d2,thr = 0.9,image_neurons=Cn)

In [ ]:
import matplotlib as mpl
mpl.use('TKAgg')
cse.utilities.view_patches_bar(Yr,coo_matrix(A_or),C_or,b2,f2, d1,d2, YrA=YrA[srt,:])
pl.show(block=True)

***REMEMBER TO STOP THE CLUSTER !!!!***

In [ ]:
cse.utilities.stop_server()